# `CoreModel` usage

In this tutorial we will introduce `COBREXA`'s `CoreModel` and
`CoreModelCoupled`. We will use *E. coli*'s toy model to start with.

In [1]:
!isfile("e_coli_core.xml") &&
    download("http://bigg.ucsd.edu/static/models/e_coli_core.xml", "e_coli_core.xml")

using COBREXA

## Loading a `CoreModel`

In [2]:
model = load_model(CoreModel, "e_coli_core.xml") # we specifically want to load a CoreModel from the model file

Metabolic model of type CoreModel
sparse([8, 10, 21, 43, 50, 51, 8, 9, 6, 12  …  33, 66, 68, 72, 23, 26, 33, 72, 22, 33], [1, 1, 1, 1, 1, 1, 2, 2, 3, 3  …  93, 93, 93, 93, 94, 94, 94, 94, 95, 95], [-1.0, 1.0, -1.0, 1.0, -1.0, 1.0, 1.0, -1.0, -1.0, 1.0  …  1.0, -1.0, 1.0, -1.0, -1.0, 1.0, 1.0, -1.0, -1.0, 1.0], 72, 95)
Number of reactions: 95
Number of metabolites: 72


## Basic analysis on `CoreModel`

As before, for optimization based analysis we need to load an optimizer. Here we
will use [`Tulip.jl`](https://github.com/ds4dm/Tulip.jl) to optimize the linear
programs of this tutorial. Refer to the examples of [analysis](05a_fba.md)
and [analysis modifications](05b_fba_mods.md) for details and explanations.

In [3]:
using Tulip

dict_sol = flux_balance_analysis_dict(
    model,
    Tulip.Optimizer;
    modifications = [
        change_objective("R_BIOMASS_Ecoli_core_w_GAM"),
        change_constraint("R_EX_glc__D_e"; lb = -12, ub = -12),
        change_constraint("R_EX_o2_e"; lb = 0, ub = 0),
    ],
)

Dict{String, Float64} with 95 entries:
  "R_EX_fum_e"    => 0.0
  "R_ACONTb"      => 0.294088
  "R_GLNS"        => 0.069699
  "R_SUCOAS"      => -5.31471e-11
  "R_TPI"         => 11.7289
  "R_EX_pi_e"     => -1.00274
  "R_PPC"         => 0.781108
  "R_O2t"         => -2.65248e-17
  "R_G6PDH2r"     => 4.23316e-9
  "R_TALA"        => -0.0487648
  "R_PPCK"        => 5.66414e-11
  "R_EX_lac__D_e" => 1.86429e-10
  "R_PGL"         => 4.23316e-9
  "R_H2Ot"        => 8.2857
  "R_GLNabc"      => 0.0
  "R_EX_co2_e"    => -0.487021
  "R_EX_gln__L_e" => 0.0
  "R_EX_nh4_e"    => -1.48633
  "R_MALt2_2"     => 0.0
  ⋮               => ⋮

## Structure of `CoreModel`

`CoreModel` is optimized for analysis of models that utilizes the matrix,
linearly-algebraic "view" of the models. It stores data in a sparse format
wherever possible.

The structure contains fields that contain the expectable model elements:

In [4]:
fieldnames(CoreModel)

(:S, :b, :c, :xl, :xu, :rxns, :mets, :grrs)

In [5]:
model.S

72×95 SparseArrays.SparseMatrixCSC{Float64, Int64} with 360 stored entries:
⠀⠀⠀⠀⠀⠄⠀⠂⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢀⠁⠀⠀⠠⠀⠀⠀⠀⠀⠀⠀⢥⠆⠀⠀⠀⠀⠀⠀⠀
⣤⠁⠃⠀⠄⡀⡀⠀⢂⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢀⠀⠀⠀⢀⢀⠀⠀⠀⡀⠀⠀⠀⠀⠀
⠀⠎⠠⣀⠤⡄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠤⣀⠀⢀⠀⠀⠀⠀⠀⠄⠄⠀⠄⠢⠀⠀⠠⠀⠀
⠀⠢⠘⡈⠒⢂⠄⠀⠀⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠒⠀⢀⢀⠀⠀⣀⠀⢀⠂⠂⠀⡚⠐⠀⠀⠐⠀⠀
⠂⠀⠀⠂⡀⠎⠂⢀⠀⠁⠀⠀⠀⠀⠀⠀⠐⠀⠀⠀⠀⠀⠀⠀⠀⠐⠀⠀⠀⠐⠐⠀⠀⠀⠂⠀⠀⠐⠄⠔
⠀⠀⠀⠀⠀⠂⠀⠈⠀⠈⡀⠀⠀⠀⠀⠀⠠⣦⠐⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠶⠀⠀⠀⠀⠀⠀⠀⠂⠂
⠀⠀⠀⠀⠀⡀⠀⠀⠀⠀⠈⠄⠀⠀⠀⠀⢀⠀⠊⢲⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢀⠀⠀⠀⠀⠀⠀⠂⣀⣀
⠀⠀⠀⠀⠀⡂⠀⠀⠀⠀⠀⠈⠄⠀⠀⠀⠀⠀⠀⠀⠈⡲⣐⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⡀⠐⡀⢀⣒⡂⣒⢂⠀⠀⠀⠀⠡⠀⠀⠀⠀⢂⠀⢂⡀⣐⣒⡁⠆⣙⣀⢀⠀⠀⡀⢐⢀⣒⢀⡀⣀⠀⡀⠀
⠀⠐⠁⠈⠈⠀⠸⠈⠀⠀⠀⠀⠀⠡⠀⠀⠀⠈⠀⡈⠀⠀⠀⠁⠐⢂⣁⣈⠀⠀⠀⠀⠈⠀⠀⠁⠉⠀⠁⠀
⠆⠀⠀⠆⠆⡆⠀⠀⠀⠀⠀⠀⠀⠀⠁⠀⠀⠀⠀⢠⠆⠀⣤⢠⢠⠆⠱⢶⡆⠰⠀⠀⠀⠀⠀⠀⠀⠀⡆⠀
⠀⠀⠀⠀⠀⡀⡠⠀⠀⠀⠀⠀⠀⠀⢐⠀⠀⠀⠀⠀⠀⠁⠉⠀⠀⠀⢀⠀⠘⡄⠀⠀⠀⡀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠒⡃⠀⠁⠀⠀⠀⠀⠀⠀⠀⠄⠀⠂⢈⠀⢊⠒⠀⠀⠀⡀⠀⣀⠀⢀⢀⠀⠰⢛⢊⡀⠀⠐⠀⠀
⠀⠀⠀⠀⠀⠄⠘⠀⠀⠀⠀⠀⠀⠀⠀⠈⠀⠀⠃⠀⠀⠀⠀⢀⠀⠀⠀⠘⠀⠀⠀⠀⠀⠀⠀⢡⠀⠃⠠⠀
⠀⠀⠀⠄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠂⠀⠂⠀⠀⠀⠀⠀⠀⠂⠀⠀⠀⠀⠀⠀⠀⠀⠀⢀⠒⠲⢉⡀

Contrary to the usual implementations, the model representation does not
contain reaction coupling boudns; these can be added to any model by wrapping
it with `CoreCoupling`. You may also use the prepared
`CoreModelCoupled` to get a version of `CoreModel` with this
coupling.

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*